In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import transforms3d as t3d
import datetime
import torch
from torch.nn import functional as F

import sys
sys.path.append("../")
import curvvae_lib.train.predictive_passthrough_trainer as ppttrainer
import curvvae_lib.architecture.passthrough_vae as ptvae
import curvvae_lib.architecture.save_model as sm
import curvvae_lib.architecture.load_model as lm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-28n3uuu3 because the default path (/home/tsr42/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
/home/tsr42/anaconda3/envs/curvvaecuda/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
foodname = "banana"
foldername = f"fork_trajectory_{foodname}"
savefilename = f"{foodname}_clean_pickups"

In [3]:
train = []
training_ts = np.linspace(0,1,64)
attempt = 1
while True:
    try:
        raw_vals = np.load(f"{savefilename}/pickup_attempt{attempt}.npy")
        train.append(raw_vals.T.flatten())
    except:
        print(f"We found {attempt-1} pickup attempts")
        break
    attempt += 1

train = np.array(train).reshape(-1,7,64)
all_points = train[:,:,:]

time_shape = list(all_points.shape)
time_shape[1] = 1
# why be smart when you can be dumb
t = np.ones(time_shape)
for i in range(time_shape[2]):
    t[:,:,i] = t[:,:,i] * i / (time_shape[2] + 0.0)

all_points = np.concatenate((t, all_points), axis=1)
all_points = all_points.transpose(0,2,1)
print(all_points.shape)

# See http://localhost:8889/notebooks/scratchwork/2021-09-17%20Rotation%20Scaling.ipynb
# for why we want quaternion values to be multiplied by 0.16 when position values are in meters 
# (if the relevant distance scale of the fork is 0.08 meters, ie: 8cm).
# scaling term doesn't affect time, so don't use time in calculation
stats_reshaped = all_points.reshape(-1,8)
mean = np.mean(stats_reshaped, axis=0)
mean[0] = 0 # don't scale time
variance = np.var(stats_reshaped[:,1:], axis=0) # don't scale time
print(mean)
print(variance)
position_std = np.sqrt(np.max(variance))
print("std of: ", position_std)
position_scaling = 1/position_std
rotation_scaling = 0.16 * position_scaling

We found 155 pickup attempts
(155, 64, 8)
[ 0.          0.00227658 -0.00117697  0.0121782  -0.04935249  0.37343377
 -0.89429268 -0.01921521]
[2.63014114e-05 3.40430938e-05 1.00819967e-04 7.90561700e-03
 3.18947674e-02 7.03375426e-03 1.11414372e-02]
std of:  0.17859106200728153


In [4]:
def scale_dataset(input_points):
    points = input_points - mean
    poss = position_scaling
    rts = rotation_scaling
    points = (points * np.array((1,poss,poss,poss,rts,rts,rts,rts)))
    return points
    
def unscale_dataset(input_points):
    poss = position_scaling
    rts = rotation_scaling
    points = (input_points / np.array((1,poss,poss,poss,rts,rts,rts,rts)))
    points = points + mean
    return points

In [5]:
scaled_points = scale_dataset(all_points)
print(np.mean(scaled_points.reshape(-1,8),axis=0))
print(np.var(scaled_points.reshape(-1,8),axis=0))

[ 4.92187500e-01 -5.33852457e-17 -3.53827630e-17 -7.10732996e-17
 -8.46335211e-17 -1.59323160e-16 -9.38040178e-16  9.83756087e-17]
[0.08331299 0.00082463 0.00106736 0.00316102 0.00634536 0.0256
 0.00564557 0.00894256]


In [36]:
import os
foldname = f"pcamodels"
os.makedirs(foldname,exist_ok=True)
testname = f"{foldname}/{foodname}_"

In [37]:
latentdim=3
savedir  = f'{testname}lat{latentdim}_pca_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
print(savedir)

pcamodels/banana_lat3_pca_20220725-203021


In [38]:
print(scaled_points.shape)
flattened_points = scaled_points[:,:,1:].reshape(scaled_points.shape[0],-1)
print(flattened_points.shape) # datapoints x (timepoints * joints)

(155, 64, 8)
(155, 448)


In [39]:
mean = np.mean(flattened_points,axis=0)
shifted = flattened_points - mean
u,s,vt = np.linalg.svd(shifted)
pca_components = s.reshape(-1,1)[:latentdim,:] * vt[:latentdim,:]
pca_components = pca_components.T
print(pca_components.shape) # want to save a (timepoints * joints) X latent_dim matrix

(448, 3)


In [40]:
print(mean.shape)

(448,)


In [41]:
np.savez(savedir, pca_components=pca_components, mean=mean)

In [42]:
pca_components = np.load(savedir+".npz")

In [45]:
pca_components["mean"]
pca_components["pca_components"]

array([[-7.82996324e-02,  1.52278060e-01,  8.37531082e-02],
       [-7.66342814e-02,  1.49588043e-01, -1.26693667e-01],
       [-2.28803008e-02,  3.62535873e-02, -9.27493532e-02],
       ...,
       [-1.58172882e+00,  2.17297025e-01, -3.71657808e-03],
       [-6.04536978e-01,  1.51692883e-03,  1.64236569e-01],
       [ 2.91450099e-01,  2.42176374e-01, -7.82171167e-01]])